# SFv2 Walkthrough

Author: Henrique F. Carvalho (2021)

## *Import module*

In [1]:
#Import modules
import pyemma
import os
import glob
import pandas as pd
import numpy as np
import sys
import importlib
import matplotlib.pyplot as plt
#Path were classes are stored
base_path='/media/dataHog/hca/'
sys.path.append(base_path)
sys.path.append(base_path+'proLig/')

import main
#import Trajectory
import Discretize
import Featurize
import plots
import tools

import warnings
warnings.filterwarnings('ignore')

importlib.reload(main)


<module 'main' from '/media/dataHog/hca/proLig/main.py'>

## **Project and system definition**

A project is defined using the ***Project*** module. Defining a *project* spawns all the simulation *systems* (with the help of ***Systems*** generator class).

The file structure and file names of the *project* are generated under the **workdir** using a hierarquical nomenclature that is used for system setup, simulation, and analysis. The **results** folder defines where results from the analysis will be outputed.

The ***Project*** module accept an arbitrary number of combination of elements, and generates a tree structure of the Cartesian product of ordered set of elements in *hierarchy* (in its current implementation, can be changed to other combinatronics schemes). The limitation resides on the required input files upon simulation execution for all the requested combinations.

For a protein-ligand simulation setup, with variation of some atribute of a ligand, the elements then become **protein(s)**, **ligand(s)** and ligand **parameter(s)**. 

*Note:* Other parameters can be coded.

In [2]:
workdir=base_path+'proLig_CalB-Methanol'
results=workdir+"/project_results"

#Check for project folders and creates them if not found)
tools.Functions.fileHandler([workdir, results], confirmation=False)

protein=['calb']
ligand=['MeOH']
parameters=['50mM', '150mM', '300mM', '600mM', '1M', '2.5M', '5.5M']

            
            
project=main.Project(title='CalB-Methanolysis', hierarchy=('protein', 'ligand', 'parameter'), workdir=workdir, 
                     parameter=parameters, replicas=10, timestep=5,
                     protein=protein, ligand=ligand, results=results)


project_systems=project.setSystems()

Elements: [['calb'], ['MeOH'], ['50mM', '150mM', '300mM', '600mM', '1M', '2.5M', '5.5M'], ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']]


## **Featurization**


Trajectories first need to be featurized with the ***Featurized*** module. Two methods are available and both are based strictly on distance metrics (angles, dihedrals, others can be coded):

1. **distance** 
2. $d_{NAC}$ (**nac**).

Both methods accept an arbitrary number of **distances** and outputs a dataframe that can be used for discretization.

The **nac** method monitors all pairwise protein-methanol distances ($d_{NAC}$) throughout simulated time. It is based on the *Near Attack Conformation* concept, applied to describe catalytic reactions.

The selected protein-substrate atom pairs are: 
1. Acetyl carbon of Ser105 (SEA, index 1526) and hydroxyl oxygen of methanol (OA).
2. Imidazole nitrogen from His224 (NE2, resid 224) and hydroxyl hydrogen of methanol (HO).

<img src="dNAC_scheme_2D.png" alt="Drawing" style="width: 300px;" align="center"/>

In [ ]:
importlib.reload(main)
importlib.reload(Featurize)


featurized={}

#distances=[('name OA', 'index 1526'), 
#      ('resname MeOH and name HO', 'resid 224 and name NE2')]

distances=[('index 1526', 'resid 224 and name NE2')]

#features['distance1']=main.Featurize(project_systems, results=results).distance(distances[0], processes=12)

featurized['dNAC']=main.Featurize(project_systems, results=results).nac(distances, feature_name='dNAC', n_cores=14)

Performing <function Featurize.nac_calculation at 0x7f822077b560> tasks for 70 elements on 14 logical cores
	Distance 1: (40000, 1, 1), (3.932, 3.932) (0.0 s)	Distance 1: (40000, 1, 1), (3.857, 3.857) (0.0 s)	Distance 1: (40000, 1, 1), (4.007, 4.007) (0.0 s)	Distance 1: (40000, 1, 1), (3.712, 3.712) (0.0 s)	Distance 1: (40000, 1, 1), (3.988, 3.988) (0.0 s)	Distance 1: (40000, 1, 1), (3.715, 3.715) (0.0 s)	Distance 1: (40000, 1, 1), (3.621, 3.621) (0.0 s)	Distance 1: (40000, 1, 1), (3.769, 3.769) (0.01 s)
	Distance 1: (40000, 1, 1), (3.824, 3.824) (0.01 s)	Distance 1: (40000, 1, 1), (3.866, 3.866) (0.01 s)
	Distance 1: (40000, 1, 1), (3.966, 3.966) (0.01 s)

	Distance 1: (40000, 1, 1), (3.741, 3.741) (0.01 s)


	Distance 1: (40000, 1, 1), (3.909, 3.909) (0.01 s)
	Distance 1: (40000, 1, 1), (3.674, 3.674) (0.01 s)Calculating d_NAC of calb-MeOH-50mM-7Calculating d_NAC of calb-MeOH-150mM-1Calculating d_NAC of calb-MeOH-50mM-5Calculating d_NAC of calb-MeOH-300mM-1Calculating d_NAC of calb-M

## **Discretization**

The *features* are discretized using the *Discretize* module. Two discretization schemes are currently available, **Uniform spherical shell discretization** and **Combinatorial user-defined spherical shell discretization**.
The *Discretize* object stores an internal copy of the provided feature **data** (DataFrame) and **feature name** (optional) to avoid passing project_system calls. Therefore, the module can take an externally-generated DataFrame, so long as the index information is consistent with the module operation.

In [ ]:
importlib.reload(main)
importlib.reload(Discretize)
importlib.reload(tools)

discretized_features={}

for name, data in featurized.items():
    discretized_features[name]=main.Discretize(data, feature_name=name, results=results)

#### **Uniform spherical shell discretization**

Discretization based in spherical shells generates spherical shells along a *feature* coordinate (*i.e.* if 1D).
The module takes as input the **thickness** of the spherical shells and the **limits** of shell edges. Discretization can also be made for different subsets of the original *feature* DataFrame, by providing the controls of the **start**, **stop** and **stride** parameters.

Returns a *shell profile* DataFrame, containing the sampled frequency for each spherical shell (index) and for each feature.

**Note:** Subset time requires the user to check what are the frames present in the *discretized* (original *featurized*) DataFrames.

In [ ]:
thickness=0.5
limits=(0,150)

shell_profiles={}

stride=1
stride=1
#times={'mini':(0,10), 
#       '2nd_half':(20000,40000), 
#       '1st_half':(0,20000), 
#       'full':(0,40000), 
#       'middle': (10000, 30000)}

times={'2nd_half':(20000,40000)}


for name, data in discretized_features.items():
    print(name)
    for time, values in times.items():
        (start_t, stop_t)=values
        shell_profiles[f'{name}-{time}']=data.shell_profile(thickness=thickness, 
                                                start=start_t, 
                                                stop=stop_t,
                                                stride=stride,
                                                limits=limits, 
                                                n_cores=14)
shell_profiles

#### **Combinatorial user-defined spherical shell discretization**

Discretize, for each frame, which combinations of shells are occupied by all ligands in the system (one state per frame)

The concept of regions is applied to the 1D $d_{NAC}$ reaction coordinate, whereby a region is defined as a *spherical shell* along the $d_{NAC}$ radius

Set the shell boundaries (limits, thickness). The minimum (0) and maximum (max) value are implicit.

E.g.: shells (4.5, 10, 12, 24) corresponds to: 
1. shell [0, 4.5[ (A)
2. shell [4.5, 10[ (P)
3. shell [10, 12[ (E)
4. shell [10, 24[ (S)
5. shell [24, max[ (B)


Labels *labels* of each region are optional but recommended (otherwise reverts to numerical). Number of *labels* is plus one the number of defined *regions*.

In [ ]:

importlib.reload(main)
shells=[4.5, 10, 12, 24]
labels=['A', 'P', 'E', 'S', 'B'] 

combinatorial={}

for name, data in discretized.items():
    combinatorial[name]=data.combinatorial(shells, labels=labels)

    print(combinatorial[name])

    discretized[name].plot(combinatorial[name])
    #TODO: remove self reference

#### **Binding free energy profile**

Represent the feaurization as a binding free energy profile (**$\Delta$G**). Options to describe the profile are *mean*, *quantile*.

In [ ]:
importlib.reload(main)
importlib.reload(tools)
importlib.reload(Discretize)
descriptors=('mean','quantile')
dG={}
bulk=(30,41)

#For quantile control
quantiles=[0.1, 0.25, 0.5, 0.75, 0.90, 0.95, 0.99, 0.999]

for name, profile in shell_profiles.items():
    print(name)
    for descriptor in descriptors:
        dG[f'{name}-{descriptor}']=main.Discretize.dG_calculation(profile, describe=descriptor,
                                             bulk=bulk,
                                             level=2,
                                             feature_name=name, 
                                             mol=ligand,
                                             quantiles=quantiles,
                                             results=results)
    

## **Markov State Models**

The *discretized* objects are used for MSM generation using the ***MSM*** module.
Each MSM is identified by the corresponding discretization *scheme*, the *feature* and *parameter* as a key of *msm_dict*. The corresponding values themselves a dictionary holding the calculated properties of the MSM. The instance of *classes.MSM* is *instance*.


In [ ]:
msms={}

for name, data in disc.items():
            msms[name]=main.MSM(data, feature_name=feature_name, feature_name, results=results)
            


**Implied Time Scales**

Calculate the Implied Time Scales for each MSM object using the *ITS* function. This function uses the pyEMMA function (pyemma.msm.its). The default array of lags can be used by pyEMMA, but here the values are provided by *lags*.

The function checks for previously calculated ITS plots by checking the presence of the corresponding *png* file in the *results* folder. 

The file path to the generated ITS image is stored in the *msm_dict* (2nd position)

In [ ]:
lags=[1, 2, 5, 10, 20, 40, 100, 500, 1000, 2000, 5000]

for name, model in msm_dict.items():
    model['its']=model['instance'].ITS(lags)
            

**Bayesian MSM**
Calculation of Bayesian MSM is made here. A lag time must be defined for each discretization *scheme-feature-parameter* combination.

In [ ]:
importlib.reload(base)
lag={}

minimum={}
minimum['nac']={'50mM':500, '150mM':1000, '300mM':1000, '600mM':2000, '1M':1000, '2.5M':2000, '5.5M':2000}
minimum['dist']={'50mM':1000, '150mM':1000, '300mM':2000, '600mM':2000, '1M':2000, '2.5M':2000, '5.5M':2000}
lag['minimum']=minimum

single={}
single['nac']={'50mM':None, '150mM':None, '300mM':None, '600mM':None, '1M':200, '2.5M':None, '5.5M':2000}
single['dist']={'50mM':None, '150mM':None, '300mM':None, '600mM':None, '1M':1000, '2.5M':None, '5.5M':None}
lag['single']=single

combinatorial={}
combinatorial['nac']={'50mM':200, '150mM':2000, '300mM':1000, '600mM':5000, '1M':2000, '2.5M':2000, '5.5M':2000}
combinatorial['dist']={'50mM':200, '150mM':200, '300mM':200, '600mM':500, '1M':500, '2.5M':1000, '5.5M':1000}
lag['combinatorial']=combinatorial

for name, model in msm_dict.items():
    print(name)
    model['bayesMSM']=model['instance'].bayesMSM(lag)

**Stationary Distribution**

Stationary distribution is calculated for each available model. Model instance accessed by *model(2)*

In [ ]:
pi=pd.DataFrame()

for name, model in msm_dict.items():
    print(name)
    pi_model=model['instance'].stationaryDistribution(model['bayesMSM'])
    pi=pd.concat([pi, pi_model], axis=1)

pi.to_csv(f'{results}/stationary_distributions.csv')

for scheme, v in disc.items():
    for feature in v.keys(): 
        if scheme == 'combinatorial':
            statdist_df=pi.xs((scheme, feature), axis=1).dropna(how='all')
            statdist_df.plot(kind='bar')
            label_names=base.Functions.sampledStateLabels(regions, sampled_states=statdist_df.index.values, labels=labels)
            positions=np.arange(0, len(label_names))
            plt.xticks(positions, label_names, rotation=70)
            plt.xlabel('Combinatorial States')
        else:
            pi.xs((scheme, feature), axis=1).plot(linestyle='-', marker='o') #kind='bar', stacked=True)
            plt.xlabel('State Index')
        plt.yscale('log')
        plt.ylabel('Stationary Distribution')
        plt.title(f'Discretization: {scheme}\nFeature: {feature}')
        plt.savefig(f'{results}/stationary_distribution-{scheme}-{feature}.png', bbox_inches='tight', dpi=600)
        plt.show()

**Flux**

Calculate the flux between set of source states *A_source* and set of sink states *B_sink*. 

In [ ]:
importlib.reload(base)
importlib.reload(plots)

flux_df=pd.DataFrame()
committor_df=pd.DataFrame()
pathway_df=pd.DataFrame()

for name,model in msm_dict.items():
    flux_model, committor_model, pathway_model=base.MSM.flux(name, model['bayesMSM'], parameter_scalar=parameter_scalar, 
                                regions=regions, labels=labels, 
                                A_source=['B', 'SB', 'ESB'],
                                B_sink=['AB', 'ASB', 'AEB', 'AESB', 'APB', 'APSB', 'APEB', 'APESB'],
                                top_pathways=3)
        
    flux_df=pd.concat([flux_df, flux_model], axis=0)
    committor_df=pd.concat([committor_df, committor_model], axis=0)
    pathway_df=pd.concat([pathway_df, pathway_model], axis=0)

    
plots.plot_pathways(pathway_df, ligand, results)
plots.plot_flux(flux_df, ligand, results)
plots.plot_committor(committor_df, ligand, results, regions, labels)


pathway_df.to_csv(f'{results}/pathways.csv')
flux_df.to_csv(f'{results}/net_fluxes.csv') 
committor_df.to_csv(f'{results}/committors.csv')





**Mean First Passage Times**

Calculate the MFPTs between all states.

k$_{on}$=1/MFPTxC, where C is concentration in Molar \
k$_{off}$=1/MFPT


Stores the resutls in the *mfpt* dataframe

**Warning**: This step takes a long time to perform. The following cell loads a pre-calculated *mfpt* dataframe.

In [ ]:
importlib.reload(base)
mfpt_df=pd.DataFrame()

for name, model in msm_dict.items():
    print(name)
    mfpt=model['instance'].MFPT(model['bayesMSM'])
    mfpt_df=pd.concat([mfpt_df, mfpt], axis=0)

mfpt_df.to_csv(f'{results}/mfpt.csv')



In [ ]:
importlib.reload(base)
importlib.reload(plots)

mfpt_df=pd.read_csv(f'{results}/mfpt.csv', index_col=[0,1,2], header=[0,1,2])

schemes=mfpt_df.index.unique(level='Scheme')
features=mfpt_df.index.unique(level='feature')
states=mfpt_df.index.unique(level='state_source')
parameters=mfpt_df.columns.get_level_values(level='parameter').unique()

error=0.2

for scheme in schemes:
    print('Scheme: ', scheme)
    for feature in features:
        print('Feature: ', feature)
        plots.plot_MFPT(mfpt_df, scheme, feature, concentrations, error=error, labels=labels, regions=regions)
        plt.savefig(f'{results}/mfpt_{scheme}-{feature}-{error}.png', bbox_inches='tight', dpi=600)
        plt.show()


## **Trajectory operations**

Initiate the trajectory object, which will be used to make further actions.

In [ ]:
importlib.reload(main)
importlib.reload(Trajectory)
importlib.reload(tools)
trajectories=main.Trajectory.Trajectory(project_systems, results=results)

### **Extract state frames**

Define the frames belonging to a set of states across all *trajectories*. A dataframe of discretized trajectories must be given. 

In [ ]:

#stateLabels=['SB', 'ESB', 'PB', 'PSB', 'PEB', 'PESB', 'AB', 'ASB', 'AEB', 'AESB', 'APSB', 'APESB']
#states=[3, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 31]

#stateLabels=['SB', 'ESB', 'PB', 'PSB', 'PESB', 'AB', 'ASB', 'APSB', 'APESB']
#states=[3,7,9,11,15,17,19,27,31]

stateLabels=['PSB']
states=[11]

extracted_frames=trajectories.extractFrames_by_iterable(combinatorial, iterable=states, feature='nac')

## Density Maps

NOTE: Compute intensive, using previously stored files.

In [ ]:
full_densities=trajectories.DensityMap_fullTraj(level=2, filtered=True, stride=5)

In [ ]:
full_densities

In [ ]:
#of extracted frames
densities, stats=trajectories.DensityMap_frames(frames=extracted_frames, level=2, dists=[('resid 290 and name NZ', 'resid 145 and name OD1')])